## 3 - Setting Up the RL Agent Training Environment

In this notebook, I go over my design for the Blackjack Gymnasium environment. [Gymnasium](https://gymnasium.farama.org/) (originally OpenAI Gym) is an open-source API for creating simulation environments to train, test, and optimize different reinforcement learning algorithms. Although Gymnasium has their own Blackjack environment, I chose to create my own environment due to having a different set of rules and possible actions. There were 3 specific methods I needed to define in order to create the class definition, including `__init__()`, `reset()`, and `step()`. For a more in-depth look at the code, check out [environment](../src/environment.py).


In [13]:
import sys
sys.path.append('../src')

from environment import BlackjackEnv
from tqdm import tqdm

## Reinforcement Learning Model

<img src="https://editor.analyticsvidhya.com/uploads/496302.jpg" alt="Blackjack basic strategy chart"  width="600" /> <br>
*Image credit: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/02/introduction-to-reinforcement-learning-for-beginners/)*

The standard model for reinforcement learning tasks consists of 2 parts: the agent and the environment. This notebook goes over the creation of the environment, while the next notebook tackles the creation of the agent. The reinforcement learning cycle goes as following:
1) The environment sends its current state to the agent (AKA observation)
2) The agent then sends the environment an action to take
3) The environment carries out the action and sends the new state back to the agent, along with any rewards gained by taking that specific action

This cycle allows reinforcement learning algorithms to learn using the "carrot and stick" method. When the agent completes a beneficial action, it receives a positive reward which encourages that behavior in the future. And when the agent completes a hurtful action, it receives a negative reward, or a punishment, which reduces their likelihood to pick that action again. Through a series of receiving states, taking actions, and getting rewards, an RL agent is able to "learn" information about what actions to take in different states to produce the best outcome.

### `__init__()` Function

For a gymnasium environment's initialization function, the 3 main things I needed to define were the action space, the observation space, and any environment variables that would be used.

The action space consists of the possible actions which can be taken by our agent. I opted to make only 3 possible actions, which were hit, stand, and double down. I decided to forego the option of splitting, as I didn't want the agent's decisions with their first hand to affect their actions on their second hand, or vice versa. Another important consideration is that although the action space consists of 3 actions, only hitting and standing were allowed after the first turn. As a result, I needed to create an "action mask" which I will clarify later.

The observation space consists of two separate observations, all the distinct types of hands we want our agent to track, as well as every possible dealer up-card. The dealer's possible up-cards go from 2 all the way through Ace, except for the fact that Jacks, Queens, and Kings are all valued at 10. So we get 10 states for the dealer, going from 2-10 plus an Ace. As for the distinct hands that we want to track, I decided to create a table that resembled the Basic Strategy guide in the second notebook. Since we aren't dealing with splits in this environment, I chose not to include pairs in the possible distinct hands. This left all the card totals from 4 (lowest possible 2 + 2) up to 20 (highest possible without busting or getting a Blackjack hand). Additionally, we have all the possible Ace hands from Ace and 2 up to Ace and 9 (excluding Ace 10 and Ace pair since the first would be a Blackjack and the second would be a pair). This leaves us with 17 card totals + 8 Ace hands = 25 total distinct card hands.

The final piece required was to define some environment variables. For my environment, this consisted of instantiating and shuffling a shoe of decks, as well as creating a variable used to track higher payouts, which I will explain later in the 4th notebook.

In [18]:
env = BlackjackEnv()

### `reset()` Function

The purpose of a gymnasium environment's reset function is to reset our environment before the start of every episode, which in our case is before every round of Blackjack. This function reinstantiates the shoe of decks if it goes below the deck penetration limit, and also creates and deals to the player's and dealer's hands.

Furthermore, resetting our environment creates 3 variables, the first depicting whether we are on our first action (to disallow doubling down after the initial deal), the second tracking whether the round is done, and the third checking if our agent chose to double down so we can double the bet amount. Finally, the function returns the observed state of the environment and any useful information, both of which will be explained later.

In [19]:
print(env.reset())

((11, 5), {'action_mask': array([1, 1, 1], dtype=int8)})


### `step()` Function

The step function is more or less straightforward, as it's purpose is to simulate a move in our round of Blackjack. It takes an action from the agent as an input, and depending on the action, it manipulates the environment variables such as the player's hand and whether the round is done or not.

In the situation that the round is complete, the function then simulates the dealer's turn using the same function from the first notebook, and then decides on a reward for the player based on if they won or lost.

At the end, this function returns the observed state of the environment, the reward earned by the turn, whether the round is completed or terminated, and any useful information. On actions where the round isn't completed, the reward returned is 0, as the agent hasn't won or lost anything.

In [21]:
print(env.step(0))

((13, 5), 0, False, False, {'action_mask': array([1, 1, 0], dtype=int8)})


### Other Functions

Whenever the environment returns the current observed state, it calls the `_get_obs()` function. This function calls 2 functions of its own, the first of which is `_get_hand_num()`. This function assigns each of the 25 possible hands to a value between 0-24 so the agent can distinguish between states. The second function is `_get_dealer_num()` which assigns the 10 possible dealer up-card values to a number between 0-9.

Another important function is `_action_mask()`. This function creates an "action mask" based on the environment variable that tracks whether the agent is currently playing their first move or not. The purpose of this function is to disallow the agent from making any illegal actions (doubling down after first turn) by limiting their action space. This may have an effect on inhibiting the learning process, but it is the best we can do without simply truncating the round and punishing our agent for taking an illegal action.

In [22]:
print(env._get_obs())

(13, 5)


In [23]:
print(env._action_mask())

[1 1 0]


### Environment Test

Now we can try running the environment over multiple episodes using an agent. In this attempt, our "agent" will simply pick a legal action at random from the action space, so there will be no "learning" occurring.

In [24]:
new_env = BlackjackEnv()

total_rewards = 0
episodes = 100000

for _ in tqdm(range(episodes)):
    # Variable "info" contains action mask
    _, info = new_env.reset()

    while not new_env.done:
        action = new_env.action_space.sample(info["action_mask"])  # The "agent"

        _, reward, _, _, info = new_env.step(action)
        total_rewards += reward
        
avg_reward = total_rewards / episodes  # Take mean of all rewards
print(f'{avg_reward * 100:.2f}')

100%|██████████| 100000/100000 [00:02<00:00, 45545.95it/s]

-48.02


As one can see, our agent tends to perform pretty horribly when we're just picking random actions. On average, we are losing 48 dollars per every 100 dollar bet, giving the casino a 48% edge over us. In the next part, we will create an agent that actually takes the rewards into account and learns from their observations.